### Bonus Analysis ;)
thought i would check another api to see if the weather stations match. Turns out theres a lot more stations than what is present in the buienradar api.

In [1]:
import requests
import pandas as pd

API_TOKEN = "eyJvcmciOiI1ZTU1NGUxOTI3NGE5NjAwMDEyYTNlYjEiLCJpZCI6ImE1OGI5NGZmMDY5NDRhZDNhZjFkMDBmNDBmNTQyNjBkIiwiaCI6Im11cm11cjEyOCJ9" 
headers = {"Authorization": f"Bearer {API_TOKEN}"}

# Step 1: Get the list of files
url = "https://api.dataplatform.knmi.nl/open-data/v1/datasets/waarneemstations_csv/versions/1.0/files"
resp = requests.get(url, headers=headers)
resp.raise_for_status()
files = resp.json()["files"]

# Pick a filename (first file)
filename = files[0]["filename"]

# Step 2: Get the download URL for this file
download_url_endpoint = f"https://api.dataplatform.knmi.nl/open-data/v1/datasets/waarneemstations_csv/versions/1.0/files/{filename}/url"
download_resp = requests.get(download_url_endpoint, headers=headers)
download_resp.raise_for_status()
download_url = download_resp.json()["temporaryDownloadUrl"]

# Step 3: Download the CSV file
csv_resp = requests.get(download_url)
with open(filename, "wb") as f:
    f.write(csv_resp.content)

# Step 4: Load and preview the CSV
df = pd.read_csv(filename, sep=",")  # adjust sep if needed
df.head()

,STN,WSI,STARTT,STOPT,LOCATIE,TYPE,HOOGTE,POS_X,POS_Y,POS_NB,POS_OL
0,201,0-20000-0-06201,2009-02-10,9999-12-31,D15-FA-1,Platform/AWS,42.70,-4.51,707.29,54.3256,2.9358
1,203,0-20000-0-06203,1990-01-01,9999-12-31,P11-B,Platform/AWS,41.84,15.67,487.76,52.3600,3.3417
2,204,0-20000-0-06204,1976-12-25,9999-12-31,K14-FA-1C,Platform/AWS,41.80,37.62,588.42,53.2694,3.6278
3,205,0-20000-0-06205,2009-02-10,9999-12-31,A12-CPP,Platform/AWS,48.35,55.02,825.23,55.3992,3.8103
4,206,0-20000-0-06206,1979-12-25,2020-09-01,F16-A,Platform/AWS,43.40,65.07,682.16,54.1167,4.0122


In [2]:
import folium

lat_col = 'POS_NB'  
lon_col = 'POS_OL'  
name_col = 'STN' 

# Create a folium map centered on the Netherlands
m = folium.Map(location=[52.2, 5.3], zoom_start=7)

# Add marker for each station
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row[lat_col], row[lon_col]],
        radius=12,  # Increase this for bigger dots
        color="#2562a6",
        fill=True,
        fill_color="#60a5fa",
        fill_opacity=0.85,
        popup=str(row[name_col])
    ).add_to(m)

# Display the map in the notebook
m